# Query ODE for products intersecting bounding-box

[ODE's REST manual]: http://oderest.rsl.wustl.edu/ODE_REST_V2.1.pdf

From [ODE's REST manual], sections 3, 3.1, we learn we can query ODE 
for products intersecting a bounding-box through the following attributes:

* `loc`: limit search by location
  * `f`: Input bounding box / footprint intersects a product’s footprint (Default)
  
* `minlat`: I'd say to limit latitudes to [-65:65] to skip poles for the time being
* `maxlat`: I'd say to limit latitudes to [-65:65] to skip poles for the time being
* `westlon`: [0:360], positive east
* `eastlon`: [0:360], positive east

And from [ODE's REST manual], page 20:
> Notes:
> 1) A requested bounding box is not valid unless all parameters (the minlat, maxlat, westernlon, and easterlon) are given. If any one of the parameters is missing, the requested bounding box is ignored.
> 2) If a requested bounding box or a requested footprint is given and no “loc” parameter is given, the default to “loc=f”
> 3) All latitudes are planetocentric
> 4) All longitude are positive east 0-360
> 5) Bounding boxes may cross the meridian
> 6) A bounding box may be a point or line but will not return results if loc=o
> See section 3.3.1 for examples of both bounding box and footprint queries with different loc= parameters.

* `proj`
  * `c0`: Cylindrical Center Lon 0 (-180 to 180)
  * `c180`: Cylindrical Center Lon 180 ( 0 to 360)
  
> About coordinate systems in ODE: https://ode.rsl.wustl.edu/mars/pagehelp/quickstartguide/index.html

* `ihid`: Instrument host ID
  * mro
  * mex
  * mgs
  
* `pt`: Product type
  * RDRV11

* `iid`
  * `hirise`

* `query`: Return list of products with selected product metadata that meet the query parameters
  * `product`

* `results`
  * `f`: Product files
  * `p`: PDS Product Identifies
  * `m`: Product metadata
  * `c`: Count products

* `output`: query output format, XML or JSON (default XML)
  * `JSON`

Examples:

* Query for (count of) HiRISE products intersecting a bounding-box:
  ```
  http://oderest.rsl.wustl.edu/live2?query=products&target=mars&results=c&ihid=MRO&iid=HiRISE&pt=RDRV11&minlat=0.0&maxlat=10.0&westernlon=358&easternlon=5&loc=f
  ```

## Ask for a bouding box crossing the frame used by ODE [0:360], planetocentric

In [1]:
bbox = {
        'minlat': -0.5,
        'maxlat': 0.5,
        'westlon': 359.5,
        'eastlon': 0.5
    }

In [2]:
import ode
from importlib import reload
reload(ode)

<module 'ode' from '/Users/chbrandt/Coisas/repos/NEANIAS/notebooks/ode.py'>

In [3]:
req = ode.request_products(ode.API_URL, bbox=bbox)

In [4]:
req.status_code

200

In [5]:
req.json()

{'ODEResults': {'Error': 'ODE V2.1.3 - ODE Rest requires either a combination of IHID, IID, and PT query parameters or PDSId query parameter or ODEID query parameter',
  'QuerySummary': {'Date': '2020-09-14T08:26:19.831',
   'easternlon': '0.5',
   'loc': 'f',
   'maxlat': '0.5',
   'minlat': '-0.5',
   'output': 'JSON',
   'query': 'PRODUCT',
   'Query_Elapsed_Time': '00:00:00.0156119',
   'results': 'FMPC',
   'westernlon': '359.5'},
  'Status': 'ERROR'}}

In [6]:
req.json()['ODEResults']['Status']

'ERROR'

In [7]:
req = ode.request_products(ode.API_URL, bbox=bbox, target='mars', host='mro', instr='hirise', ptype='rdrv11')

req.json()['ODEResults']['Status'] == 'ERROR' and req.json() or req.json()['ODEResults']['Count'] + ' products found'

'6 products found'

In [8]:
req.url

'https://oderest.rsl.wustl.edu/live2/?query=product&results=fmpc&output=JSON&loc=f&minlat=-0.5&maxlat=0.5&westlon=359.5&eastlon=0.5&target=mars&ihid=mro&iid=hirise&pt=rdrv11'

In [9]:
req.json()

{'ODEResults': {'Count': '6',
  'Products': {'Product': [{'BB_georeferenced': 'True',
     'Center_georeferenced': 'True',
     'Center_latitude': '-0.012',
     'Center_longitude': '0.007',
     'Comment': 'Center of Mars',
     'Data_Set_Id': 'MRO-M-HIRISE-3-RDR-V1.1',
     'Description': 'HiRISE projected and mosaicked product  HiRISE RDR V1.1 files has map projection data embedded in the header - please see SIS for more details.',
     'Easternmost_longitude': '0.044',
     'Emission_angle': '0.172008',
     'External_url': 'http://www.uahirise.org/PSP_007361_1800',
     'External_url2': 'jpip://hijpip.lpl.arizona.edu:8064/PDS/RDR/PSP/ORB_007300_007399/PSP_007361_1800/PSP_007361_1800_COLOR.JP2',
     'FilesURL': 'http://ode.rsl.wustl.edu/mars/productfiles.aspx?product_id=PSP_007361_1800_COLOR&product_idGeo=14284086',
     'Footprint_C0_geometry': 'POLYGON ((0.044 -0.237, 0.025 -0.239, -0.03 0.212, -0.011 0.214, 0.044 -0.237))',
     'Footprint_geometry': 'GEOMETRYCOLLECTION (POLYGO

### Read out product results

In [10]:
list(req.json()['ODEResults'].keys())

['Count', 'Products', 'QuerySummary', 'Status']

In [11]:
list(req.json()['ODEResults']['Products'].keys())

['Product']

In [12]:
type(req.json()['ODEResults']['Products']['Product'])

list

In [13]:
len(req.json()['ODEResults']['Products']['Product'])

6

In [14]:
req.json()['ODEResults']['Products']['Product'][0]

{'BB_georeferenced': 'True',
 'Center_georeferenced': 'True',
 'Center_latitude': '-0.012',
 'Center_longitude': '0.007',
 'Comment': 'Center of Mars',
 'Data_Set_Id': 'MRO-M-HIRISE-3-RDR-V1.1',
 'Description': 'HiRISE projected and mosaicked product  HiRISE RDR V1.1 files has map projection data embedded in the header - please see SIS for more details.',
 'Easternmost_longitude': '0.044',
 'Emission_angle': '0.172008',
 'External_url': 'http://www.uahirise.org/PSP_007361_1800',
 'External_url2': 'jpip://hijpip.lpl.arizona.edu:8064/PDS/RDR/PSP/ORB_007300_007399/PSP_007361_1800/PSP_007361_1800_COLOR.JP2',
 'FilesURL': 'http://ode.rsl.wustl.edu/mars/productfiles.aspx?product_id=PSP_007361_1800_COLOR&product_idGeo=14284086',
 'Footprint_C0_geometry': 'POLYGON ((0.044 -0.237, 0.025 -0.239, -0.03 0.212, -0.011 0.214, 0.044 -0.237))',
 'Footprint_geometry': 'GEOMETRYCOLLECTION (POLYGON ((360 -0.034, 359.97 0.212, 359.989 0.214, 360 0.124, 360 -0.034)), POLYGON ((0.044 -0.237, 0.025 -0.239, 0

In [15]:
help(ode)

Help on module ode:

NAME
    ode

FUNCTIONS
    find_product_file(product_files, product_type, descriptors={'ctx': {'product_image': ('Description', 'PRODUCT DATA FILE WITH LABEL'), 'browse_image': ('Description', 'BROWSE IMAGE'), 'browse_thumbnail': ('Description', 'THUMBNAIL IMAGE')}, 'hirise': {'product_image': ('Description', 'PRODUCT DATA FILE'), 'product_label': ('Description', 'PRODUCT LABEL FILE'), 'browse_image': ('Description', 'BROWSE'), 'browse_thumbnail': ('Description', 'THUMBNAIL')}})
    
    readout_product_files(product_json)
    
    readout_product_footprint(request)
    
    readout_product_meta(product_json)
    
    request_product(PRODUCTID, api_endpoint)
    
    request_products(api_endpoint, bbox, target=None, host=None, instr=None, ptype=None)
        bbox = {
            'minlat': [-65:65],
            'minlat': [-65:65],
            'westlon': [0:360],
            'eastlon': [0:360]
        }
        'ptype' (eg, "rdrv11") is used only when 'instr' is als

In [23]:
reload(ode)
products = ode.requested_products(req)

In [24]:
len(products)

6

In [25]:
i = 0
pi = products[i]

imeta = ode.readout_product_meta(pi)
ifiles = ode.readout_product_files(pi)
ifprint = ode.readout_product_footprint(pi)

In [26]:
imeta

{'id': 'PSP_007361_1800_COLOR',
 'mission': 'MRO',
 'inst': 'HIRISE',
 'type': 'RDRV11'}

In [27]:
ifiles

[{'Description': 'MAP PROJECTION FILE',
  'FileName': 'DSMAP.CAT',
  'KBytes': '7',
  'Type': 'Referenced',
  'URL': 'https://hirise.lpl.arizona.edu/PDS/CATALOG/DSMAP.CAT'},
 {'Description': 'DESCRIPTION FILE',
  'FileName': 'JP2INFO.TXT',
  'KBytes': '9',
  'Type': 'Referenced',
  'URL': 'https://hirise.lpl.arizona.edu/PDS/DOCUMENT/JP2INFO.TXT'},
 {'Description': 'ANNOTATED BROWSE',
  'FileName': 'PSP_007361_1800_COLOR.ABROWSE.JPG',
  'KBytes': '1036',
  'Type': 'Browse',
  'URL': 'https://hirise.lpl.arizona.edu/PDS/EXTRAS/RDR/PSP/ORB_007300_007399/PSP_007361_1800/PSP_007361_1800_COLOR.abrowse.jpg'},
 {'Description': 'BROWSE',
  'FileName': 'PSP_007361_1800_COLOR.BROWSE.JPG',
  'KBytes': '1191',
  'Type': 'Browse',
  'URL': 'https://hirise.lpl.arizona.edu/PDS/EXTRAS/RDR/PSP/ORB_007300_007399/PSP_007361_1800/PSP_007361_1800_COLOR.browse.jpg'},
 {'Description': 'PRODUCT DATA FILE',
  'FileName': 'PSP_007361_1800_COLOR.JP2',
  'KBytes': '189587',
  'Type': 'Product',
  'URL': 'https://hi

In [28]:
ifprint

'POLYGON ((0.044 -0.237, 0.025 -0.239, 359.97 0.212, 359.989 0.214, 0.044 -0.237))'

In [30]:
ode.find_product_file(product_files=ifiles, product_type='product_image', descriptors=ode.DESCRIPTORS['hirise'])

{'Description': 'PRODUCT DATA FILE',
 'FileName': 'PSP_007361_1800_COLOR.JP2',
 'KBytes': '189587',
 'Type': 'Product',
 'URL': 'https://hirise.lpl.arizona.edu/PDS/RDR/PSP/ORB_007300_007399/PSP_007361_1800/PSP_007361_1800_COLOR.JP2'}

## Condensed workflow

In [92]:
reload(ode)

INSTRUMENT = 'hirise'

req = ode.request_products(ode.API_URL, bbox=bbox, target='mars', host='mro', instr=INSTRUMENT, ptype='rdrv11')

products = ode.requested_products(req)

products_output = []
for i,product in enumerate(products):
    _meta = ode.readout_product_meta(product)
    _files = ode.readout_product_files(product)
    _fprint = ode.readout_product_footprint(product)
    _pfile = ode.find_product_file(product_files=_files, product_type='product_image', descriptors=ode.DESCRIPTORS[INSTRUMENT])
    _pfile = _pfile['URL']
    _lfile = ode.find_product_file(product_files=_files, product_type='product_label', descriptors=ode.DESCRIPTORS[INSTRUMENT])
    _lfile = _lfile['URL']
    _dout = _meta
    _dout['geometry'] = _fprint
    _dout['image_url'] = _pfile
    _dout['label_url'] = _lfile
    products_output.append(_dout)

print(products_output)

[{'id': 'PSP_007361_1800_COLOR', 'mission': 'MRO', 'inst': 'HIRISE', 'type': 'RDRV11', 'geometry': 'POLYGON ((0.044 -0.237, 0.025 -0.239, 359.97 0.212, 359.989 0.214, 0.044 -0.237))', 'image_url': 'https://hirise.lpl.arizona.edu/PDS/RDR/PSP/ORB_007300_007399/PSP_007361_1800/PSP_007361_1800_COLOR.JP2', 'label_url': 'https://hirise.lpl.arizona.edu/PDS/RDR/PSP/ORB_007300_007399/PSP_007361_1800/PSP_007361_1800_COLOR.LBL'}, {'id': 'PSP_007361_1800_RED', 'mission': 'MRO', 'inst': 'HIRISE', 'type': 'RDRV11', 'geometry': 'POLYGON ((0.08 -0.234, 359.989 -0.245, 359.935 0.206, 0.025 0.216, 0.08 -0.234))', 'image_url': 'https://hirise.lpl.arizona.edu/PDS/RDR/PSP/ORB_007300_007399/PSP_007361_1800/PSP_007361_1800_RED.JP2', 'label_url': 'https://hirise.lpl.arizona.edu/PDS/RDR/PSP/ORB_007300_007399/PSP_007361_1800/PSP_007361_1800_RED.LBL'}, {'id': 'ESP_023817_1800_COLOR', 'mission': 'MRO', 'inst': 'HIRISE', 'type': 'RDRV11', 'geometry': 'POLYGON ((359.942 0, 359.923 -0.002, 359.899 0.202, 359.918 0.2

In [93]:
import geopandas as gpd
import shapely

for prod in products_output:
    prod['geometry'] = shapely.wkt.loads(prod['geometry'])

gdf = gpd.GeoDataFrame(products_output)

gdf.to_file('bla.geojson', driver='GeoJSON')

In [94]:
!cat 'bla.geojson'

{
"type": "FeatureCollection",
"features": [
{ "type": "Feature", "properties": { "id": "PSP_007361_1800_COLOR", "mission": "MRO", "inst": "HIRISE", "type": "RDRV11", "image_url": "https:\/\/hirise.lpl.arizona.edu\/PDS\/RDR\/PSP\/ORB_007300_007399\/PSP_007361_1800\/PSP_007361_1800_COLOR.JP2", "label_url": "https:\/\/hirise.lpl.arizona.edu\/PDS\/RDR\/PSP\/ORB_007300_007399\/PSP_007361_1800\/PSP_007361_1800_COLOR.LBL" }, "geometry": { "type": "Polygon", "coordinates": [ [ [ 0.044, -0.237 ], [ 0.025, -0.239 ], [ 359.97, 0.212 ], [ 359.989, 0.214 ], [ 0.044, -0.237 ] ] ] } },
{ "type": "Feature", "properties": { "id": "PSP_007361_1800_RED", "mission": "MRO", "inst": "HIRISE", "type": "RDRV11", "image_url": "https:\/\/hirise.lpl.arizona.edu\/PDS\/RDR\/PSP\/ORB_007300_007399\/PSP_007361_1800\/PSP_007361_1800_RED.JP2", "label_url": "https:\/\/hirise.lpl.arizona.edu\/PDS\/RDR\/PSP\/ORB_007300_007399\/PSP_007361_1800\/PSP_007361_1800_RED.LBL" }, "geometry": { "type": "Polygon", "coordinates": [

## Make it a function

Query footprints intersecting given bounding-box; Footprints for a particular instrument.

In [110]:
import ode

def search_footprints(bbox, dataset):
    """
    Dataset be like: 'mro/hirise/rdrv11'
    """
    host,instr,ptype = dataset.split('/')
    
    req = ode.request_products(ode.API_URL, bbox=bbox, target='mars', host=host, instr=instr, ptype=ptype)
    
    products = ode.requested_products(req)
    
    products_output = []
    for i,product in enumerate(products):
        _meta = ode.readout_product_meta(product)
        _files = ode.readout_product_files(product)
        _fprint = ode.readout_product_footprint(product)
        _pfile = ode.find_product_file(product_files=_files, product_type='product_image', descriptors=ode.DESCRIPTORS[INSTRUMENT])
        _pfile = _pfile['URL']
        _lfile = ode.find_product_file(product_files=_files, product_type='product_label', descriptors=ode.DESCRIPTORS[INSTRUMENT])
        _lfile = _lfile['URL']
        _dout = _meta
        _dout['geometry'] = _fprint
        _dout['image_url'] = _pfile
        _dout['label_url'] = _lfile
        products_output.append(_dout)

    return products_output


def products_to_geojson(products_output, filename):
    import geopandas as gpd
    import shapely

    for prod in products_output:
        try:
            prod['geometry'] = shapely.wkt.loads(prod['geometry'])
        except TypeError as err:
            print("Error in: ", prod)
#             raise err
        
    gdf = gpd.GeoDataFrame(products_output)

    gdf.to_file(filename, driver='GeoJSON')
    print("File '{}' written.".format(filename))


In [111]:
print(bbox)

{'minlat': -0.5, 'maxlat': 0.5, 'westlon': 359.5, 'eastlon': 0.5}


In [112]:
# define query
bbox = {'minlat': -0.5, 'maxlat': 0.5, 'westlon': 359.5, 'eastlon': 0.5}
dataset = 'mro/hirise/rdrv11'

_dset = dataset.replace('/','-')
_bbox = ','.join(str(val) for val in bbox.values())

filename = '{dset}_{bbox}.geojson'.format(dset=_dset,bbox=_bbox)

# query products
products = search_footprints(bbox=bbox, dataset=dataset)

In [113]:
# import json
# print(json.dumps(products, indent=2))
products

[{'id': 'PSP_007361_1800_COLOR',
  'mission': 'MRO',
  'inst': 'HIRISE',
  'type': 'RDRV11',
  'geometry': 'POLYGON ((0.044 -0.237, 0.025 -0.239, 359.97 0.212, 359.989 0.214, 0.044 -0.237))',
  'image_url': 'https://hirise.lpl.arizona.edu/PDS/RDR/PSP/ORB_007300_007399/PSP_007361_1800/PSP_007361_1800_COLOR.JP2',
  'label_url': 'https://hirise.lpl.arizona.edu/PDS/RDR/PSP/ORB_007300_007399/PSP_007361_1800/PSP_007361_1800_COLOR.LBL'},
 {'id': 'PSP_007361_1800_RED',
  'mission': 'MRO',
  'inst': 'HIRISE',
  'type': 'RDRV11',
  'geometry': 'POLYGON ((0.08 -0.234, 359.989 -0.245, 359.935 0.206, 0.025 0.216, 0.08 -0.234))',
  'image_url': 'https://hirise.lpl.arizona.edu/PDS/RDR/PSP/ORB_007300_007399/PSP_007361_1800/PSP_007361_1800_RED.JP2',
  'label_url': 'https://hirise.lpl.arizona.edu/PDS/RDR/PSP/ORB_007300_007399/PSP_007361_1800/PSP_007361_1800_RED.LBL'},
 {'id': 'ESP_023817_1800_COLOR',
  'mission': 'MRO',
  'inst': 'HIRISE',
  'type': 'RDRV11',
  'geometry': 'POLYGON ((359.942 0, 359.923 

In [114]:
# geojson products
print("Writing {}".format(filename))
products_to_geojson(products, filename)

Writing mro-hirise-rdrv11_-0.5,0.5,359.5,0.5.geojson
File 'mro-hirise-rdrv11_-0.5,0.5,359.5,0.5.geojson' written.


In [115]:
!cat $filename

{
"type": "FeatureCollection",
"features": [
{ "type": "Feature", "properties": { "id": "PSP_007361_1800_COLOR", "mission": "MRO", "inst": "HIRISE", "type": "RDRV11", "image_url": "https:\/\/hirise.lpl.arizona.edu\/PDS\/RDR\/PSP\/ORB_007300_007399\/PSP_007361_1800\/PSP_007361_1800_COLOR.JP2", "label_url": "https:\/\/hirise.lpl.arizona.edu\/PDS\/RDR\/PSP\/ORB_007300_007399\/PSP_007361_1800\/PSP_007361_1800_COLOR.LBL" }, "geometry": { "type": "Polygon", "coordinates": [ [ [ 0.044, -0.237 ], [ 0.025, -0.239 ], [ 359.97, 0.212 ], [ 359.989, 0.214 ], [ 0.044, -0.237 ] ] ] } },
{ "type": "Feature", "properties": { "id": "PSP_007361_1800_RED", "mission": "MRO", "inst": "HIRISE", "type": "RDRV11", "image_url": "https:\/\/hirise.lpl.arizona.edu\/PDS\/RDR\/PSP\/ORB_007300_007399\/PSP_007361_1800\/PSP_007361_1800_RED.JP2", "label_url": "https:\/\/hirise.lpl.arizona.edu\/PDS\/RDR\/PSP\/ORB_007300_007399\/PSP_007361_1800\/PSP_007361_1800_RED.LBL" }, "geometry": { "type": "Polygon", "coordinates": [

In [116]:
print(INSTRUMENT)

hirise
